In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw4.ipynb")

In [1]:
# Setup
# You may import math if needed in your solutions.
import math
import numpy as np
import scipy as sp
from dataclasses import dataclass

## **Question 1**: Using Objects

In [2]:
# CLASS DEFINITION - Use this class in your solution below
class SimplePendulum:
    """
    A class representing a simple pendulum for physics calculations.
    """
    
    def __init__(self, length, mass, gravity=9.81):
        """
        Initialize a simple pendulum.
        
        Parameters:
        - length: length of the pendulum string (meters)
        - mass: mass of the pendulum bob (kg)
        - gravity: acceleration due to gravity (m/s^2, default 9.81)
        """
        self.length = length
        self.mass = mass
        self.gravity = gravity
        self.angle = 0.0  # current angle in radians
        self.angular_velocity = 0.0  # current angular velocity in rad/s
    
    def set_initial_conditions(self, initial_angle, initial_angular_velocity=0.0):
        """
        Set the initial angle and angular velocity of the pendulum.
        
        Parameters:
        - initial_angle: initial angle in radians
        - initial_angular_velocity: initial angular velocity in rad/s (default 0)
        """
        self.angle = initial_angle
        self.angular_velocity = initial_angular_velocity
    
    def natural_frequency(self):
        """
        Calculate the natural frequency of small oscillations.
        
        Returns:
        - frequency in Hz (cycles per second)
        """
        omega = math.sqrt(self.gravity / self.length)
        return omega / (2 * math.pi)
    
    def period(self):
        """
        Calculate the period of small oscillations.
        
        Returns:
        - period in seconds
        """
        return 2 * math.pi * math.sqrt(self.length / self.gravity)
    
    def potential_energy(self):
        """
        Calculate the gravitational potential energy at current angle.
        Uses the reference point at the lowest position of the swing.
        
        Returns:
        - potential energy in Joules
        """
        height = self.length * (1 - math.cos(self.angle))
        return self.mass * self.gravity * height
    
    def kinetic_energy(self):
        """
        Calculate the kinetic energy at current angular velocity.
        
        Returns:
        - kinetic energy in Joules
        """
        linear_velocity = self.angular_velocity * self.length
        return 0.5 * self.mass * linear_velocity**2
    
    def total_energy(self):
        """
        Calculate the total mechanical energy (kinetic + potential).
        
        Returns:
        - total energy in Joules
        """
        return self.kinetic_energy() + self.potential_energy()

### Pendulum Energy Analysis

Using the `SimplePendulum` class defined above, write a function `analyze_pendulum_swing(length, mass, max_angle, num_positions=50)` that analyzes the energy distribution during a complete swing.

The function should:
1. **Create a pendulum object** using the given length and mass
2. **Simulate swing positions**: Create an array of angles from `-max_angle` to `+max_angle` with `num_positions` evenly spaced points
3. **Calculate energy at each position**: For each angle, set the pendulum to that position with zero angular velocity, then calculate:
   - Potential energy at that position
   - Kinetic energy needed to maintain total energy conservation (assuming the pendulum started from rest at `max_angle`)
4. **Find energy conservation properties**:
   - Total mechanical energy (should be constant)
   - Maximum kinetic energy (occurs at bottom of swing)
   - Maximum potential energy (occurs at maximum displacement)

Parameters:
- `length`: pendulum length in meters
- `mass`: pendulum mass in kg  
- `max_angle`: maximum swing angle in radians
- `num_positions`: number of positions to analyze (default 50)

Return a tuple: `(angles, potential_energies, kinetic_energies, total_energy, max_kinetic, max_potential)`
- `angles`: numpy array of angle positions (radians)
- `potential_energies`: numpy array of potential energies at each position (Joules)
- `kinetic_energies`: numpy array of kinetic energies at each position (Joules)  
- `total_energy`: single value of total mechanical energy (Joules)
- `max_kinetic`: maximum kinetic energy in the swing (Joules)
- `max_potential`: maximum potential energy in the swing (Joules)

**Hint**: Use `np.linspace()` to create the angle array and use the pendulum's methods to calculate energies.

In [3]:
def analyze_pendulum_swing(length, mass, max_angle, num_positions=50):
    # BEGIN SOLUTION NO PROMPT
    # Create a pendulum object
    pendulum = SimplePendulum(length, mass)
    
    # Create array of angles from -max_angle to +max_angle
    angles = np.linspace(-max_angle, max_angle, num_positions)
    
    # Calculate total energy (from maximum angle with zero velocity)
    pendulum.set_initial_conditions(max_angle, 0.0)
    total_energy = pendulum.potential_energy()  # KE = 0 at max angle
    
    # Initialize arrays for energies
    potential_energies = np.zeros(num_positions)
    kinetic_energies = np.zeros(num_positions)
    
    # Calculate energies at each position
    for i, angle in enumerate(angles):
        # Set pendulum to current angle with zero angular velocity
        pendulum.set_initial_conditions(angle, 0.0)
        
        # Calculate potential energy at this position
        pe = pendulum.potential_energy()
        potential_energies[i] = pe
        
        # Calculate kinetic energy from conservation: KE = Total - PE
        kinetic_energies[i] = total_energy - pe
    
    # Find maximum energies
    max_kinetic = np.max(kinetic_energies)
    max_potential = np.max(potential_energies)
    
    return (angles, potential_energies, kinetic_energies, total_energy, max_kinetic, max_potential)
    # END SOLUTION
    """ # BEGIN PROMPT
    # Write your code here!
    return (angles, potential_energies, kinetic_energies, total_energy, max_kinetic, max_potential)
    """; # END PROMPT

In [ ]:
grader.check("q1")

## **Question 2**: Creating a Spring-Mass System Class

In this problem, you will create your own class to model a spring-mass system, one of the fundamental systems in physics. 

Create a class called `SpringMassSystem` that models a mass attached to a spring. Your class should:

**Constructor `__init__(self, mass, spring_constant, damping_coefficient=0.0)`:**
- `mass`: mass of the object (kg)
- `spring_constant`: spring constant k (N/m) 
- `damping_coefficient`: damping coefficient b (kg/s), default 0.0 for no damping
- Initialize `position` and `velocity` attributes to 0.0

**Methods to implement:**

1. **`set_initial_conditions(self, position, velocity=0.0)`**
   - Set the initial position (m) and velocity (m/s) of the mass

2. **`natural_frequency(self)`**
   - Return the natural angular frequency ω₀ = √(k/m) in rad/s
   - For undamped systems only

3. **`period(self)`**
   - Return the period T = 2π/ω₀ in seconds
   - For undamped systems only

4. **`potential_energy(self)`**
   - Return the elastic potential energy: PE = ½kx² in Joules
   - Use the current position

5. **`kinetic_energy(self)`**
   - Return the kinetic energy: KE = ½mv² in Joules  
   - Use the current velocity

6. **`total_energy(self)`**
   - Return the total mechanical energy (KE + PE) in Joules

Write your class definition below:

In [9]:
class SpringMassSystem:
    # BEGIN SOLUTION NO PROMPT
    """
    A class representing a spring-mass system for physics calculations.
    """
    
    def __init__(self, mass, spring_constant, damping_coefficient=0.0):
        """
        Initialize a spring-mass system.
        
        Parameters:
        - mass: mass of the object (kg)
        - spring_constant: spring constant k (N/m)
        - damping_coefficient: damping coefficient b (kg/s), default 0.0
        """
        self.mass = mass
        self.spring_constant = spring_constant
        self.damping_coefficient = damping_coefficient
        self.position = 0.0  # current position in meters
        self.velocity = 0.0  # current velocity in m/s
    
    def set_initial_conditions(self, position, velocity=0.0):
        """
        Set the initial position and velocity of the mass.
        
        Parameters:
        - position: initial position in meters
        - velocity: initial velocity in m/s (default 0)
        """
        self.position = position
        self.velocity = velocity
    
    def natural_frequency(self):
        """
        Calculate the natural angular frequency for undamped oscillations.
        
        Returns:
        - angular frequency ω₀ = √(k/m) in rad/s
        """
        return math.sqrt(self.spring_constant / self.mass)
    
    def period(self):
        """
        Calculate the period for undamped oscillations.
        
        Returns:
        - period T = 2π/ω₀ in seconds
        """
        return 2 * math.pi / self.natural_frequency()
    
    def potential_energy(self):
        """
        Calculate the elastic potential energy at current position.
        
        Returns:
        - potential energy PE = ½kx² in Joules
        """
        return 0.5 * self.spring_constant * self.position**2
    
    def kinetic_energy(self):
        """
        Calculate the kinetic energy at current velocity.
        
        Returns:
        - kinetic energy KE = ½mv² in Joules
        """
        return 0.5 * self.mass * self.velocity**2
    
    def total_energy(self):
        """
        Calculate the total mechanical energy (kinetic + potential).
        
        Returns:
        - total energy in Joules
        """
        return self.kinetic_energy() + self.potential_energy()
    # END SOLUTION
    """ # BEGIN PROMPT
    # Write your SpringMassSystem class here!
    """; # END PROMPT

In [ ]:
grader.check("q2")

## **Question 3**: Creating a Normal Distribution Class

Create a class called `NormalDistribution` that models a normal distribution with a given mean and variance. Your class should:

**Constructor `__init__(self, mean, variance)`:**
- `mean`: the mean μ of the distribution
- `variance`: the variance σ² of the distribution 
- Store these as instance attributes

**Methods to implement:**

1. **`pdf(self, x)`**
   - Return the probability density function (PDF) evaluated at all points in array `x`
   - The Gaussian PDF is: $f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$
   - Should work with both single values and numpy arrays

2. **`cdf(self, x)`**
   - Return the cumulative distribution function (CDF) evaluated at all points in array `x`  
   - The Gaussian CDF is: $F(x) = \frac{1}{2}\left[1 + \text{erf}\left(\frac{x-\mu}{\sigma\sqrt{2}}\right)\right]$
   - Use `scipy.special.erf` for the error function
   - Should work with both single values and numpy arrays

**Mathematical Background:**
- The standard deviation σ is the square root of the variance: σ = √(σ²)
- The error function `erf(z)` is available as `scipy.special.erf(z)`

Write your class definition below:

In [17]:
class NormalDistribution:
    # BEGIN SOLUTION NO PROMPT
    """
    A class representing a normal (Gaussian) distribution.
    """
    
    def __init__(self, mean, variance):
        """
        Initialize a normal distribution.
        
        Parameters:
        - mean: the mean μ of the distribution
        - variance: the variance σ² of the distribution
        """
        self.mean = mean
        self.variance = variance
        self.std_dev = math.sqrt(variance)  # Standard deviation σ
    
    def pdf(self, x):
        """
        Calculate the probability density function at points x.
        
        Parameters:
        - x: single value or numpy array of values
        
        Returns:
        - PDF values at x: f(x) = (1/√(2πσ²)) * exp(-(x-μ)²/(2σ²))
        """
        # Convert to numpy array for vectorized operations
        x = np.asarray(x)
        
        # Calculate normalization factor
        normalization = 1.0 / math.sqrt(2 * math.pi * self.variance)
        
        # Calculate exponential term
        exponent = -0.5 * ((x - self.mean) ** 2) / self.variance
        
        return normalization * np.exp(exponent)
    
    def cdf(self, x):
        """
        Calculate the cumulative distribution function at points x.
        
        Parameters:
        - x: single value or numpy array of values
        
        Returns:
        - CDF values at x: F(x) = 0.5 * [1 + erf((x-μ)/(σ√2))]
        """
        # Convert to numpy array for vectorized operations
        x = np.asarray(x)
        
        # Calculate argument for error function
        erf_arg = (x - self.mean) / (self.std_dev * math.sqrt(2))
        
        # Use scipy.special.erf for error function
        return 0.5 * (1 + sp.special.erf(erf_arg))
    # END SOLUTION
    """ # BEGIN PROMPT
    # Write your NormalDistribution class here!
    """; # END PROMPT

In [ ]:
grader.check("q3")

## **Question 4**: Error Handling

Write a function `safe_relativistic_kinetic_energy(mass, velocity, c=299792458)` that calculates the relativistic kinetic energy using the formula:

$$KE_{rel} = (\gamma - 1)mc^2$$

where $\gamma = \frac{1}{\sqrt{1 - \frac{v^2}{c^2}}}$ is the Lorentz factor.

**Your function must implement comprehensive error handling using try/except/else/finally blocks:**

**Required Error Checks:**
1. **Type validation**: All inputs must be numbers (int or float)
2. **Physical constraints**: 
   - Mass must be positive (m > 0)
   - Velocity must be non-negative and less than the speed of light (0 ≤ v < c)
   - Speed of light must be positive (c > 0)
3. **Mathematical issues**: Handle potential division by zero or domain errors

**Return behavior:**
- **Success**: Return the relativistic kinetic energy in Joules
- **Any error**: Return the string `"Error: [descriptive error message]"`

**Structure requirements:**
- Use `try` block for the main calculation
- Use `except` blocks to catch and handle different types of errors 

**Parameters:**
- `mass`: rest mass in kg
- `velocity`: velocity in m/s  
- `c`: speed of light in m/s (default: 299,792,458 m/s)

Write your function below:
Hint: your except blocks can look like:  
    `except TypeError:`  
        &emsp;`return "Error: Invalid data types for calculation"`  
    `except ValueError as e:`  
        &emsp;`return f"Error: Mathematical error - {str(e)}"`  
    `except ZeroDivisionError:`  
        &emsp;`return "Error: Division by zero in calculation"`  
    `except OverflowError:`  
        &emsp;`return "Error: Number too large for calculation"`  
    `except Exception as e:`  
        &emsp;`return f"Error: Unexpected error - {str(e)}"`  

In [26]:
def safe_relativistic_kinetic_energy(mass, velocity, c=299792458):
    # BEGIN SOLUTION NO PROMPT
    """
    Calculate relativistic kinetic energy with comprehensive error handling.
    
    Parameters:
    - mass: rest mass in kg
    - velocity: velocity in m/s
    - c: speed of light in m/s
    
    Returns:
    - Relativistic kinetic energy in Joules, or error message string
    """
    
    try:
        # Type validation
        if not isinstance(mass, (int, float)) or not isinstance(velocity, (int, float)) or not isinstance(c, (int, float)):
            return "Error: All inputs must be numbers (int or float)"
        
        # Physical constraint validation
        if mass <= 0:
            return "Error: Mass must be positive"
        
        if velocity < 0:
            return "Error: Velocity must be non-negative"
            
        if velocity >= c:
            return "Error: Velocity must be less than speed of light"
            
        if c <= 0:
            return "Error: Speed of light must be positive"
        
        # Calculate relativistic kinetic energy
        # First calculate v²/c²
        velocity_ratio_squared = (velocity / c) ** 2
        
        # Calculate Lorentz factor γ = 1/√(1 - v²/c²)
        denominator = 1 - velocity_ratio_squared
        
        # Check for mathematical issues (should not happen given our constraints, but good practice)
        if denominator <= 0:
            return "Error: Mathematical domain error in Lorentz factor calculation"
            
        gamma = 1 / math.sqrt(denominator)
        
        # Calculate relativistic kinetic energy: KE = (γ - 1)mc²
        kinetic_energy = (gamma - 1) * mass * c**2
        
    except TypeError:
        return "Error: Invalid data types for calculation"
    except ValueError as e:
        return f"Error: Mathematical error - {str(e)}"
    except ZeroDivisionError:
        return "Error: Division by zero in calculation"
    except OverflowError:
        return "Error: Number too large for calculation"
    except Exception as e:
        return f"Error: Unexpected error - {str(e)}"
    
    return kinetic_energy
    # END SOLUTION
    """ # BEGIN PROMPT
    # Write your safe_relativistic_kinetic_energy function here!
    pass
    """; # END PROMPT

In [ ]:
grader.check("q4")

## **Question 5**: N-Body Simulation with Dataclasses

In computational physics, N-body simulations track the positions and momenta of multiple particles over time. Python's `@dataclass` decorator provides a clean way to define classes that primarily hold data.

In this problem, you will create a dataclass to represent a snapshot of an N-body simulation and write a function to analyze the system's properties.

**Part 1: Create the dataclass**

Create a dataclass called `NBodySnapshot` using the `@dataclass` decorator with the following attributes:

- `positions`: numpy array of shape (N, 3) representing 3D positions of N particles (in meters)
- `momenta`: numpy array of shape (N, 3) representing 3D momenta of N particles (in kg⋅m/s)  
- `masses`: numpy array of shape (N,) representing masses of N particles (in kg)
- `time`: float representing the simulation time (in seconds)

**Part 2: Create the analysis function**

Write a function `analyze_system(snapshot)` that takes an `NBodySnapshot` object and returns system properties:

- **Average position**: The center of mass position vector (3D numpy array)
  $$\vec{r}_{cm} = \frac{1}{M} \sum_{i=1}^{N} m_i \vec{r}_i$$
  where M is the total mass of the system

- **Average velocity**: The center of mass velocity vector (3D numpy array)  
  $$\vec{v}_{cm} = \frac{1}{M} \sum_{i=1}^{N} m_i \vec{v}_i = \frac{1}{M} \sum_{i=1}^{N} \vec{p}_i$$
  where $\vec{p}_i$ is the momentum of particle i

**Return format**: The function should return a tuple `(avg_position, avg_velocity)` where both are numpy arrays of shape (3,).

Note: Use vectorized numpy operations for efficiency

Hint: numpy array datatype is given by `np.ndarray`

Write your dataclass and function below:

In [36]:
@dataclass
class NBodySnapshot:
    # BEGIN SOLUTION NO PROMPT
    """
    A dataclass representing a snapshot of an N-body simulation.
    """
    positions: np.ndarray  # Shape (N, 3) - 3D positions in meters
    momenta: np.ndarray    # Shape (N, 3) - 3D momenta in kg⋅m/s  
    masses: np.ndarray     # Shape (N,) - masses in kg
    time: float            # Simulation time in seconds

def analyze_system(snapshot):
    """
    Analyze an N-body system snapshot to compute center of mass properties.
    
    Parameters:
    - snapshot: NBodySnapshot object
    
    Returns:
    - tuple (avg_position, avg_velocity) where both are 3D numpy arrays
    """
    # Calculate total mass of the system
    total_mass = np.sum(snapshot.masses)
    
    # Calculate center of mass position (mass-weighted average)
    # r_cm = (1/M) * sum(m_i * r_i)
    mass_weighted_positions = snapshot.positions * snapshot.masses[:, np.newaxis]  # Broadcasting
    avg_position = np.sum(mass_weighted_positions, axis=0) / total_mass
    
    # Calculate center of mass velocity
    # v_cm = (1/M) * sum(p_i) = (1/M) * sum(m_i * v_i)
    # Since p_i = m_i * v_i, we can use momenta directly
    avg_velocity = np.sum(snapshot.momenta, axis=0) / total_mass
    
    return (avg_position, avg_velocity)
# END SOLUTION

""" # BEGIN PROMPT
# Write your NBodySnapshot dataclass and analyze_system function here!

#can comment out below while developing dataclass
def analyze_system(snapshot):
    
    return (avg_position, avg_velocity)
"""; # END PROMPT

In [ ]:
grader.check("q5")

## Required disclosure of use of AI technology

Please indicate whether you used AI to complete this homework. If you did, explain how you used it in the python cell below, as a comment.

In [45]:
# BEGIN SOLUTION NO PROMPT
# END SOLUTION
""" # BEGIN PROMPT
"""
# write ai disclosure here:

"""
"""; # END PROMPT

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

Upload the .zip file to Gradescope!

In [ ]:
grader.export(pdf=False, force_save=True, run_tests=True)